In [ ]:
import helper
import matplotlib.pyplot as plt
from keras.applications import *
from keras.callbacks import EarlyStopping
import os

In [ ]:
#设置各种参数
train_path = ['./data/train1/cat', './data/train1/dog']
test_path ='./data/test1/test1'
img_size =(299,299)
layer_num = 249
model_image ='./models/model_image_fine_tuning_inceptionv3_0401.png'
model_weights_file = './models/weights_fine_tuning_inceptionv3_with_outliers_0401.h5'
template_csv_path = './predicts/sample_submission.csv'
target_csv_path = './predicts/pred_fine_tuning_inceptionv3_with_outliers_0401.csv'
MODEL = inception_v3.InceptionV3
preprocess_func = inception_v3.preprocess_input

In [3]:
#获取训练集数据
X_train, y_train, image_files= helper.get_train_input_from_folder_with_subclasses(train_path, img_size, lambda_func=preprocess_func)
print("finished")

(25000, 299, 299, 3)
(25000, 1)
finished
finished


In [ ]:
#构造模型，锁定base_model所有层
model = helper.get_fine_tuning_first_model(MODEL)

#可视化模型
helper.visualize_model(model, model_image)
print("finished")

In [5]:
print("start")
#第一次训练新添加层权重
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(X_train, y_train, batch_size=128, epochs=8, validation_split=0.2, callbacks=[early_stopping])
print("finished")

start
Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 74s 4ms/step - loss: 0.2430 - acc: 0.9139 - val_loss: 0.0555 - val_acc: 0.9892
Epoch 2/8
20000/20000 [==============================] - 68s 3ms/step - loss: 0.1070 - acc: 0.9640 - val_loss: 0.1555 - val_acc: 0.9422
Epoch 3/8
20000/20000 [==============================] - 68s 3ms/step - loss: 0.0905 - acc: 0.9675 - val_loss: 0.0911 - val_acc: 0.9708
Epoch 4/8
20000/20000 [==============================] - 68s 3ms/step - loss: 0.0850 - acc: 0.9692 - val_loss: 0.0606 - val_acc: 0.9842
finished


In [6]:
print("start")
#放开若干层权重，再次训练
model = helper.get_fine_tuning_second_model(model, layer_num)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
model.fit(X_train, y_train, batch_size=128, epochs=50, validation_split=0.2, callbacks=[early_stopping])
print("finished")

start
Train on 20000 samples, validate on 5000 samples
Epoch 1/50
20000/20000 [==============================] - 76s 4ms/step - loss: 0.0752 - acc: 0.9721 - val_loss: 0.0307 - val_acc: 0.9912
Epoch 2/50
20000/20000 [==============================] - 73s 4ms/step - loss: 0.0674 - acc: 0.9759 - val_loss: 0.0279 - val_acc: 0.9912
Epoch 3/50
20000/20000 [==============================] - 73s 4ms/step - loss: 0.0639 - acc: 0.9772 - val_loss: 0.0254 - val_acc: 0.9914
Epoch 4/50
20000/20000 [==============================] - 74s 4ms/step - loss: 0.0581 - acc: 0.9785 - val_loss: 0.0240 - val_acc: 0.9920
Epoch 5/50
20000/20000 [==============================] - 74s 4ms/step - loss: 0.0554 - acc: 0.9802 - val_loss: 0.0228 - val_acc: 0.9920
Epoch 6/50
20000/20000 [==============================] - 74s 4ms/step - loss: 0.0538 - acc: 0.9803 - val_loss: 0.0217 - val_acc: 0.9924
Epoch 7/50
20000/20000 [==============================] - 74s 4ms/step - loss: 0.0496 - acc: 0.9824 - val_loss: 0.0210 - va

In [7]:
#保存模型参数
model.save_weights(model_weights_file)
del X_train
del y_train
print("finished")

finished


In [8]:
print("start")
#获取测试数据和对应文件
X_test, test_files = helper.get_input_from_folder_with_image_files(test_path, img_size, lambda_func=preprocess_func)

#获取文件basename
image_file_names = [os.path.basename(path) for path in test_files]

#预测并保存预测结果到csv
helper.predict_and_update_to_csv(model, X_test, image_file_names, template_csv_path, target_csv_path)

print("finished")

start
finished
12500/12500 [==============================] - 41s 3ms/step
finished
